# Million Song Baseline #

Baseline model to get a feel for loading the data + setting up logistic regression on the million song subset

In [1]:
import pandas as pd
import numpy as np
import tables
import seaborn as sns
#import xgboost as xgb

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline

import imblearn.over_sampling
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

from collections import OrderedDict


In [2]:
import sys
sys.version

'3.5.5 |Anaconda, Inc.| (default, Apr 26 2018, 08:11:22) \n[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]'

In [3]:
train = tables.open_file(r'E:\Nicolas\Documents\Metis\Music_Baseline\MillionSongSubset\data\train.h5')

OSError: ``E:\Nicolas\Documents\Metis\Music_Baseline\MillionSongSubset\data\train.h5`` does not exist

In [ ]:
song_metadata = train.root.metadata.songs
song_metadata_df = pd.DataFrame.from_records(song_metadata[:])
song_metadata_df.head()

In [ ]:
song_analysis = train.root.analysis.songs
song_analysis_df = pd.DataFrame.from_records(song_analysis[:])
song_analysis_df.head()

In [ ]:
full_song_data = song_metadata_df.merge(song_analysis_df, on=song_analysis_df.index)
full_song_data.head()

In [ ]:
full_song_data.columns

In [ ]:
full_song_data.idx_artist_terms[221]

In [ ]:
full_song_data = full_song_data.drop(['key_0', 'analyzer_version', 'artist_7digitalid', 'artist_playmeid',\
                                      'idx_artist_terms', 'idx_similar_artists', 'release_7digitalid', 'analysis_sample_rate',\
                                      'idx_bars_confidence', 'idx_bars_start', 'idx_beats_confidence', 'idx_beats_start',\
                                      'idx_sections_confidence', 'idx_sections_start', 'idx_segments_confidence',\
                                      'idx_segments_loudness_max', 'idx_segments_loudness_max_time', 'idx_segments_loudness_start',\
                                      'idx_segments_pitches', 'idx_segments_start', 'idx_segments_timbre', 'idx_tatums_confidence',\
                                      'idx_tatums_start', 'track_7digitalid', 'audio_md5', 'artist_id', 'artist_latitude',\
                                      'artist_longitude', 'artist_mbid', 'song_id', 'song_hotttnesss', 'artist_location', 'genre'], axis=1)
full_song_data = full_song_data.sort_values(by='danceability', ascending=False)
full_song_data.head(25)

In [ ]:
len(full_song_data[full_song_data['danceability'] == 0.0])

In [ ]:
type(full_song_data.key[0])

In [ ]:
def clean_b(value):
    if type(value) == bytes:
        no_b = str(value).lstrip('b').replace('&', 'and')
        no_quote = no_b.replace('"', '')
        return no_quote.replace("'", "").lower()
    elif type(value) == float and value == 0.0:
        return np.nan
    else: 
        return value

full_song_data = full_song_data.applymap(clean_b)
full_song_data.head()

In [ ]:
year = pd.read_csv(r'E:\Nicolas\Documents\Metis\Music_Baseline\MillionSongSubset\AdditionalFiles\subset_tracks_per_year.txt', sep='<SEP>', names=['year', 'track_id', 'artist_name', 'title'])
year.head()

In [ ]:
def lowercase(value):
    return value.lower()
year_only = year.drop(['artist_name', 'title'], axis = 1)
year_only['track_id'] = year_only['track_id'].astype(str).apply(lowercase)
full_song_data = full_song_data.merge(year_only, on='track_id')
full_song_data.shape

In [ ]:
year_only['year'].describe()

In [ ]:
full_song_data.isnull().sum()

In [ ]:
full_song_data = full_song_data.drop(['danceability', 'energy'], axis=1)
full_song_data = full_song_data.dropna()

In [ ]:
full_song_data.shape
#hot_100['title'] = hot_100['song']
#hot_100['artist_name'] = hot_100['performer']

In [ ]:
hot_100 = pd.read_csv(r'E:\Nicolas\Documents\Metis\Music_Baseline\get-top-40-hot-or-not-QueryResult.csv')
hot_100 = hot_100.rename(index=str, columns={"song": "title", "performer": "artist_name"})
hot_100['isin_hot100'] = 1
hot_100.head()

In [ ]:
def clean_ands(value):
    value = str(value).replace('"', '')
    value = value.replace("'", "")
    return value.replace('&', 'and').lower()
hot_100 = hot_100.applymap(clean_ands)
hot_100.sort_values('title').head()

In [ ]:
full_song_data.sort_values('title').head()

In [ ]:
merged_song_data = full_song_data.merge(hot_100, how='left', on=['title', 'artist_name'])
merged_song_data = merged_song_data.sort_values(by='year', ascending=True)
merged_song_data.head()

In [ ]:
merged_song_data['isin_hot100']=merged_song_data['isin_hot100'].fillna(0)
merged_song_data['isin_hot100']=merged_song_data['isin_hot100'].apply(pd.to_numeric)
merged_song_data.head()

In [ ]:
merged_song_data_clean = merged_song_data.drop(['key_confidence', 'mode_confidence', 'time_signature_confidence', 'track_id'], axis=1)
merged_song_data_clean['time_signature'] = merged_song_data_clean['time_signature'].astype(str)
merged_song_data_clean['key'] = merged_song_data_clean['key'].astype(str)
merged_song_data_clean.head()

In [ ]:
#sns.pairplot(merged_song_data_clean)

In [ ]:
merged_song_data_clean.to_csv(r'E:\Nicolas\Documents\Metis\Music_Baseline\MillionSongSubset\data\merged_song_data_clean.csv')

In [ ]:
merged_song_data_clean['year'] = merged_song_data_clean['year'] - int(1927)
merged_song_data_clean.head()

### Now let's do the same on the holdout set ###

In [ ]:
holdout = tables.open_file(r'E:\Nicolas\Documents\Metis\Music_Baseline\MillionSongSubset\data\test.h5')
holdout_metadata = holdout.root.metadata.songs
holdout_metadata_df = pd.DataFrame.from_records(holdout_metadata[:])
holdout_analysis = holdout.root.analysis.songs
holdout_analysis_df = pd.DataFrame.from_records(holdout_analysis[:])
full_holdout_data = holdout_metadata_df.merge(holdout_analysis_df, on=holdout_analysis_df.index)
full_holdout_data = full_holdout_data.drop(['key_0', 'analyzer_version', 'artist_7digitalid', 'artist_playmeid',\
                                      'idx_artist_terms', 'idx_similar_artists', 'release_7digitalid', 'analysis_sample_rate',\
                                      'idx_bars_confidence', 'idx_bars_start', 'idx_beats_confidence', 'idx_beats_start',\
                                      'idx_sections_confidence', 'idx_sections_start', 'idx_segments_confidence',\
                                      'idx_segments_loudness_max', 'idx_segments_loudness_max_time', 'idx_segments_loudness_start',\
                                      'idx_segments_pitches', 'idx_segments_start', 'idx_segments_timbre', 'idx_tatums_confidence',\
                                      'idx_tatums_start', 'track_7digitalid', 'audio_md5', 'artist_id', 'artist_latitude',\
                                      'artist_longitude', 'artist_mbid', 'song_id', 'song_hotttnesss', 'artist_location', 'genre',\
                                      'key_confidence', 'mode_confidence', 'time_signature_confidence', 'track_id', 'danceability', 'energy'], axis=1)


In [ ]:
X_data = merged_song_data_clean.loc[:,['artist_name', 'title', 'artist_familiarity', 'artist_hotttnesss', 'duration',\
                                       'end_of_fade_in', 'key', 'loudness', 'mode', 'start_of_fade_out', 'tempo',\
                                       'time_signature', 'year']]
y_data = merged_song_data_clean['isin_hot100']

# Establishing a Baseline with Logistic Regression and SMOTE #

In [ ]:
X, x_test, y, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=42)

In [ ]:
y_test.describe()

In [ ]:
selection = ['artist_hotttnesss']

In [ ]:
smote = imblearn.over_sampling.SMOTE(ratio=.4, random_state = 42) 
    
X_tr_smote, y_tr_smote = smote.fit_sample(X.loc[:,selection], y)

lr_smote = LogisticRegression() 
lr_smote.fit(X_tr_smote, y_tr_smote)
y_pred = lr_smote.predict(x_test.loc[:,selection])

print('Logistic Regression on SMOTE Train Data, ratio 0.4; Test F1: %.3f, Test AUC: %.3f' % \
      (f1_score(lr_smote.predict(x_test.loc[:,selection]), y_test), roc_auc_score(y_test, lr_smote.predict_proba(x_test.loc[:,selection])[:,1])))

In [ ]:
x_val, y_val = x_test.loc[:,selection], y_test # explicitly calling this validation since we're using it for selection

thresh_ps = np.linspace(.10,.50,1000)
model_val_probs = lr_smote.predict_proba(x_val)[:,1] # positive class probs, same basic logistic model we fit in section 2 

f1_scores = []
for p in thresh_ps:
    model_val_labels = model_val_probs >= p
    f1_scores.append(f1_score(model_val_labels, y_val))
    
plt.plot(thresh_ps, f1_scores)
plt.title('F1 Score vs. Positive Class Decision Probability Threshold')
plt.xlabel('P threshold')
plt.ylabel('F1 score')

best_f1_score = np.max(f1_scores) 
best_thresh_p = thresh_ps[np.argmax(f1_scores)]

print('Logistic Regression Model best F1 score %.3f at prob decision threshold >= %.3f' 
      % (best_f1_score, best_thresh_p))

## Let's try some more features ##

In [ ]:
selection = ['artist_familiarity', 'artist_hotttnesss', 'duration', 'key', 'loudness', 'mode', 'tempo', 'time_signature', 'year']

In [ ]:
std = StandardScaler()
x_train_cat = pd.get_dummies(X.loc[:,selection], columns=['key', 'time_signature'])
x_test_cat = pd.get_dummies(x_test.loc[:,selection], columns=['key', 'time_signature'])
x_train_scaled = std.fit_transform(x_train_cat)
x_test_scaled = std.fit_transform(x_test_cat)

smote = imblearn.over_sampling.SMOTE(ratio=.4, random_state = 42) 
    
X_tr_smote, y_tr_smote = smote.fit_sample(x_train_scaled, y)

lr_smote1 = LogisticRegression(class_weight={1 : 4, 0 : 1}) 
lr_smote2 = LogisticRegression(class_weight='balanced')
lr_smote1.fit(X_tr_smote, y_tr_smote)
lr_smote2.fit(X_tr_smote, y_tr_smote)
y_pred1 = lr_smote1.predict(x_test_scaled)
y_pred2 = lr_smote2.predict(x_test_scaled)

print('Logistic Regression on SMOTE Train Data, ratio 0.4, weight balanced; Test F1: %.3f, Test AUC: %.3f, RMSE: %.3f'  % \
      (f1_score(y_pred1, y_test), roc_auc_score(y_test, lr_smote1.predict_proba(x_test_scaled)[:,1]), np.sqrt(np.sum((y_pred1 - y_test)**2)/len(y_test))))

print('Logistic Regression on SMOTE Train Data, ratio 0.4, weight 3:1; Test F1: %.3f, Test AUC: %.3f, RMSE: %.3f' % \
      (f1_score(y_pred2, y_test), roc_auc_score(y_test, lr_smote2.predict_proba(x_test_scaled)[:,1]), np.sqrt(np.sum((y_pred2 - y_test)**2)/len(y_test))))

In [ ]:
x_val, y_val = x_test_scaled, y_test # explicitly calling this validation since we're using it for selection

thresh_ps = np.linspace(.10,1,1000)
model_val_probs = lr_smote1.predict_proba(x_val)[:,1] # positive class probs, same basic logistic model we fit in section 2 

f1_scores = []
for p in thresh_ps:
    model_val_labels = model_val_probs >= p
    f1_scores.append(f1_score(model_val_labels, y_val))
    
plt.plot(thresh_ps, f1_scores)
plt.title('F1 Score vs. Positive Class Decision Probability Threshold')
plt.xlabel('P threshold')
plt.ylabel('F1 score')

best_f1_score = np.max(f1_scores) 
best_thresh_p = thresh_ps[np.argmax(f1_scores)]

print('Logistic Regression Model best F1 score %.3f at prob decision threshold >= %.3f' 
      % (best_f1_score, best_thresh_p))

## Random Forest ##

In [ ]:
rf1 = RandomForestClassifier(n_estimators = 1000, max_features = 3,
                                min_samples_leaf = 5, n_jobs=-1, class_weight='balanced')
rf2 = RandomForestClassifier(n_estimators = 1000, max_features = 5,
                                min_samples_leaf = 5, n_jobs=-1, class_weight={1 : 4, 0 : 1})

x_smote, y_smote = smote.fit_sample(X.loc[:,selection], y)

rf1.fit(X_tr_smote, y_smote)
rf2.fit(X_tr_smote, y_smote)
y_pred1 = rf1.predict(x_test_scaled)
y_pred2 = rf2.predict(x_test_scaled)

print('Random Forest on SMOTE Train Data, ratio 0.4; Test F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_pred1, y_test), roc_auc_score(y_test, rf1.predict_proba(x_test_scaled)[:,1])))
print(rf1.feature_importances_)
print('Random Forest on SMOTE Train Data, ratio 0.4; Test F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_pred2, y_test), roc_auc_score(y_test, rf2.predict_proba(x_test_scaled)[:,1])))
print(rf2.feature_importances_)

In [ ]:
x_val, y_val = x_test_scaled, y_test # explicitly calling this validation since we're using it for selection

thresh_ps = np.linspace(.10,1,1000)
model_val_probs = rf2.predict_proba(x_val)[:,1] # positive class probs, same basic logistic model we fit in section 2 

f1_scores = []
for p in thresh_ps:
    model_val_labels = model_val_probs >= p
    f1_scores.append(f1_score(model_val_labels, y_val))
    
plt.plot(thresh_ps, f1_scores)
plt.title('F1 Score vs. Positive Class Decision Probability Threshold')
plt.xlabel('P threshold')
plt.ylabel('F1 score')

best_f1_score = np.max(f1_scores) 
best_thresh_p = thresh_ps[np.argmax(f1_scores)]

print('Logistic Regression Model best F1 score %.3f at prob decision threshold >= %.3f' 
      % (best_f1_score, best_thresh_p))

## Gradient Boosting ##

In [ ]:
gb1 = xgboost(n_estimators = 1000, learning_rate = 1, max_depth = 7)
gb1.fit(x_smote, y_smote)
y_pred1 = gb1.predict(x_test.loc[:,selection])

print('Gradient Boosting on SMOTE Train Data, ratio 0.4; Test F1: %.3f, Test AUC: %.3f' % \
      (f1_score(y_pred1, y_test), roc_auc_score(y_test, gb1.predict_proba(x_test.loc[:,selection])[:,1])))

In [ ]:
x_val, y_val = x_test.loc[:,selection], y_test # explicitly calling this validation since we're using it for selection

thresh_ps = np.linspace(.10,0.5,1000)
model_val_probs = gb1.predict_proba(x_val)[:,1]
f1_scores = []
for p in thresh_ps:
    model_val_labels = model_val_probs >= p
    f1_scores.append(f1_score(model_val_labels, y_val))
    
plt.plot(thresh_ps, f1_scores)
plt.title('F1 Score vs. Positive Class Decision Probability Threshold')
plt.xlabel('P threshold')
plt.ylabel('F1 score')

best_f1_score = np.max(f1_scores) 
best_thresh_p = thresh_ps[np.argmax(f1_scores)]

print('Logistic Regression Model best F1 score %.3f at prob decision threshold >= %.3f' 
      % (best_f1_score, best_thresh_p))

In [ ]:
gb1.plot_importance()